# Library
- 발생, 사망, 성별에서 value 부분 결측 '-', 수치 모두 문자로 되어있음.
- mapping을 이용해 결측은 np.nan으로 만들고 수치는 숫자로 만들기

- 발생의 경우 `급수` 맵핑해줘야함.
- 사망의 경우 애초에 `군구` 없음.
- 성별의 경우 `급수`, `시도` 맵핑 필요.

In [1]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from tqdm import tqdm

In [2]:
dat = pd.read_csv('성별0121_수정중_교수님틀.csv')
ori = pd.read_csv('성별2001-2021.csv')

C:\Users\kelly\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\kelly\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
dat.shape
ori.shape

(359856, 31)

(479808, 25)

In [4]:
63*272*21
dat.head() # val 열 : 28, row : 21년 * 병 종류수(63) * 군구수(272) 만큼

,연도,병,군구,0~4세_남,0~4세_여,5~9세_남,5~9세_여,10~14세_남,10~14세_여,15~19세_남,...,45~49세_남,45~49세_여,50~54세_남,50~54세_여,55~59세_남,55~59세_여,60~64세_남,60~64세_여,65세 이상_남,65세 이상_여
0,2001년,에볼라바이러스병,1,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
1,2002년,에볼라바이러스병,1,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
2,2003년,에볼라바이러스병,1,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
3,2004년,에볼라바이러스병,1,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
4,2005년,에볼라바이러스병,1,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-


In [9]:
14*2*63*272

ori.head() # row 연령단위 종류(14) * 성별 종류(2) * 병종류(63) * 군구수(272)

479808

# labeling

In [10]:
## 시도lst

시도lst = ['서울', '부산', '대구', '인천',
      '광주', '대전', '울산', '경기', '강원', 
      '충북', '충남', '전북', '전남', '경북',
      '경남', '제주', '세종']
len(시도lst) # 17

## name2병id (len 63)

급1병_lst = ['에볼라바이러스병', '마버그열', '라싸열', '크리미안콩고출혈열', '남아메리카출혈열', '리프트밸리열', '두창', '페스트',
            '탄저', '보툴리눔독소증', '야토병', '중증급성호흡기증후군(SARS)', '중동호흡기증후군(MERS)', '동물인플루엔자 인체감염증',
            '신종인플루엔자', '디프테리아']

급2병_lst = ['수두', '홍역', '콜레라', '장티푸스', '파라티푸스', '세균성이질', '장출혈성대장균감염증', 'A형간염', '백일해',
            '유행성이하선염', '풍진(2018년이전)', '풍진(선천성)', '풍진(후천성)', '폴리오', '수막구균 감염증',
            'b형헤모필루스인플루엔자', '폐렴구균 감염증', '한센병', '성홍열', '반코마이신내성황색포도알균(VRSA) 감염증',
            '카바페넴내성장내세균속균종(CRE) 감염증', 'E형간염']

급3병_lst = ['파상풍', 'B형간염', '일본뇌염', 'C형간염', '말라리아', '레지오넬라증', '비브리오패혈증', '발진티푸스', '발진열',
             '쯔쯔가무시증', '렙토스피라증', '브루셀라증', '공수병', '신증후군출혈열',
             '크로이츠펠트-야콥병(CDJ) 및 변종크로이츠펠트-야콥병(vCJD)', '황열', '뎅기열', '큐열', '웨스트나일열', '라임병',
             '진드기매개뇌염', '유비저', '치쿤구니야열', '중증열성혈소판감소증후군(SFTS)', '지카바이러스감염증']

병name_lst = 급1병_lst + 급2병_lst + 급3병_lst
len(병name_lst) # 63

병id_lst = [i for i in range(len(급1병_lst))] + [i for i in range(len(급2병_lst))] + [i for i in range(len(급3병_lst))]

name2병id = {병name_lst[i] : 병id_lst[i] for i in range(len(병id_lst))}
len(name2병id) # 63

급id_match_lst = ['1급']*len(급1병_lst) + ['2급']*len(급2병_lst) + ['3급']*len(급3병_lst)
name2급id = {병name_lst[i] : 급id_match_lst[i] for i in range(len(병id_lst))} # 8차 최종에 추가


## 지역 (len 272)
군구서울 = ['강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구', '용산구', '은평구',
           '종로구', '중구', '중랑구']

군구부산 = ['강서구', '금정구', '기장군', '남구', '동구', '동래구', '부산진구', '북구', '사상구', '사하구', '서구', '수영구', '연제구', '영도구', '중구', '해운대구']

군구대구 = ['남구', '달서구', '달성군', '동구', '북구', '서구', '수성구', '중구']

군구인천 = ['강화군', '계양구', '미추홀구', '남동구', '동구', '부평구', '서구', '연수구', '옹진군', '중구']

군구광주 = ['광산구', '남구', '동구', '북구', '서구']

군구대전 = ['대덕구', '동구', '서구', '유성구', '중구']

군구울산 = ['남구', '동구', '북구', '울주군', '중구']

군구경기 = ['가평군', '고양시 덕양구', '고양시 일산동구', '고양시 일산서구', '과천시', '광명시', '광주시', '구리시', '군포시', '김포시', '남양주시', '동두천시', '부천시 소사구', '부천시 오정구', '부천시 원미구', '성남시 분당구', '성남시 수정구',
           '성남시 중원구', '수원시 권선구', '수원시 영통구', '수원시 장안구', '수원시 팔달구', '시흥시', '안산시 단원구', '안산시 상록구', '안성시', '안양시 동안구', '안양시 만안구', '양주시', '양평군', '여주시', '연천군', '오산시', '용인시 기흥구', '용인시 수지구',
           '용인시 처인구', '의왕시', '의정부시', '이천시', '파주시', '평택시', '포천시', '하남시', '화성시', '양주군', '고양시 일산구', '포천군', '용인시', '남양주', '고양시 일산동', '안산시', '부천시']

군구강원 = ['강릉시', '고성군', '동해시', '삼척시', '속초시', '양구군', '양양군', '영월군', '원주시', '인제군', '정선군', '철원군', '춘천시', '태백시', '평창군', '홍천군', '화천군', '횡성군']

군구충북 = ['괴산군', '단양군', '보은군', '영동군', '옥천군', '음성군', '제천시',' 증평군', '진천군', '청원군', '(구)청주시 상당구', '(구)청주시 흥덕구', '충주시', '청주시 상당구', '청주시 흥덕구', '청주시 서원구', '청주시 청원구']

군구충남 = ['계룡시', '공주시', '금산군', '논산시', '당진시', '보령시', '부여군', '서산시', '서천군', '아산시', '연기군', '예산군', '천안시', '청양군', '태안군', '홍성군', '천안시 동남구', '천안시 서북구']

군구전북 = ['고창군', '군산시' ,'김제시', '남원시', '무주군', '부안군', '순창군', '완주군', '익산시', '임실군', '장수군', '전주시 덕진구', '전주시 완산구', '정읍시', '진안군', '전주시']

군구전남 = ['강진군', '고흥군', '곡성군', '광양시', '구례군', '나주시', '담양군', '목포시', '무안군', '보성군', '순천시', '신안군', '여수시', '영광군', '영암군', '완도군', '장성군', '장흥군', '진도군', '함평군', '해남군', '화순군']

군구경북 = ['경산시', '경주시', '고령군', '구미시', '군위군', '김천시', '문경시', '봉화군', '상주시', '성주군', '안동시', '영덕군', '영양군', '영주시', '영천시', '예천군', '울릉군', '울진군', '의성군', '청도군', '청송군', '칠곡군',
           '포항시 남구', '포항시 북구', '포항시']

군구경남 = ['거제시', '거창군', '고성군', '김해시', '남해군', '마산시', '밀양시', '사천시', '산청군', '양산시', '의령군', '진주시', '진해시', '창녕군', '창원시', '통영시', '하동군', '함안군', '함양군', '합천군', '창원시 마산합포구', '창원시 마산회원구',
           '창원시 성산구', '창원시 의창구', '창원시 진해구']

군구제주 = ['서귀포시', '제주시', '남제주군', '북제주군']

군구세종 = ['세종시']



### 각 시도별 군구 길이
# for i in 시도lst:
#     i, len(globals()[f'군구{i}'])


### 모든 시도의 모든 군구의 수
군구all = 군구서울 + 군구부산 + 군구대구 + 군구인천 + 군구광주 + 군구대전 + 군구울산 + 군구경기 + 군구강원 + 군구충북 + 군구충남 + 군구전북 + 군구전남 + 군구경북 + 군구경남 + 군구제주 + 군구세종
len(군구all) # 272

17

63

63

272

# Data Load 및 df 조정
- columns 위치 바꾸기 등의 간단한 조정부터 교수님께서 원하시는 shape을 만들기 위한 다소 복잡한 연산까지

In [102]:
발생 = pd.read_csv('발생2001_01-2022_06.csv')
발생 = 발생[['연도', '월', '시도', '군구']+병name_lst]

# 사망 = pd.read_csv('사망2001-2021.csv')
# 사망.rename(columns = {'군구':'시도'}, inplace = True)
# 사망 = 사망[['연도', '시도', '급수', '병']+[f'{i}월' for i in range(1, 13)]]

# 성별 = pd.read_csv('성별2001-2021.csv')
# 성별 = 성별[['병', '군구', '연령단위', '성별']+[f'{i}년' for i in range(2001, 2022)]]

# ## 컬럼 추가, 위치 지정
# 연령성별 = 성별.연령단위 + '_' + 성별.성별
# 성별.insert(2, '연령성별', 연령성별)

# ## drop
# 성별.drop(['연령단위', '성별'], axis = 1, inplace = True)

## 발생

In [31]:
발생.shape
발생.head()

(70176, 67)

,연도,월,시도,군구,에볼라바이러스병,마버그열,라싸열,크리미안콩고출혈열,남아메리카출혈열,리프트밸리열,...,황열,뎅기열,큐열,웨스트나일열,라임병,진드기매개뇌염,유비저,치쿤구니야열,중증열성혈소판감소증후군(SFTS),지카바이러스감염증
0,2001,1,서울,강남구,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2001,1,서울,강동구,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2001,1,서울,강북구,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2001,1,서울,강서구,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2001,1,서울,관악구,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [104]:
발생T = 발생.iloc[:,4:].T

발생T.reset_index(inplace = True)
발생T.rename(columns = {'index':'병'}, inplace = True)

In [107]:
발생T.head()

,병,0,1,2,3,4,5,6,7,8,...,70166,70167,70168,70169,70170,70171,70172,70173,70174,70175
0,에볼라바이러스병,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,마버그열,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,라싸열,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,크리미안콩고출혈열,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,남아메리카출혈열,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [121]:
df = pd.DataFrame()
for i in tqdm(range(발생T.shape[1])):
    tmp병 = 발생T.iloc[:,1:].iloc[:,i]
    
    tmp식별 = list(발생.iloc[i,:4].values)
    tmp식별 = pd.DataFrame([tmp식별]*len(tmp병))
    tmp식별 = pd.concat([tmp식별, 발생T.iloc[:,0]], axis = 1)
    
    tmpdf = pd.concat([tmp식별, tmp병], axis = 1)
    tmpdf.columns = ['연도', '월', '시도', '군구', '병', '발생수']
    tmpdf.index = list(range(i*63, (i+1)*63))
    
    df = pd.concat([df, tmpdf], ignore_index = True)

100%|█████████████████████████████████████████████████████████████████████████▉| 70176/70177 [3:39:57<00:00,  5.32it/s]


IndexError: single positional indexer is out-of-bounds

In [126]:
df.shape
21 * 12 * 272 * 63 + 6* 272 * 63 # 연도 * 월 * 시도 * 군구 * 병 + 월 * 시도 * 군구
df.head()
df.tail()

(4421088, 6)

4421088

,연도,월,시도,군구,병,발생수
0,2001,1,서울,강남구,에볼라바이러스병,0
1,2001,1,서울,강남구,마버그열,0
2,2001,1,서울,강남구,라싸열,0
3,2001,1,서울,강남구,크리미안콩고출혈열,0
4,2001,1,서울,강남구,남아메리카출혈열,0


,연도,월,시도,군구,병,발생수
4421083,2022,6,세종,세종시,진드기매개뇌염,0
4421084,2022,6,세종,세종시,유비저,0
4421085,2022,6,세종,세종시,치쿤구니야열,0
4421086,2022,6,세종,세종시,중증열성혈소판감소증후군(SFTS),0
4421087,2022,6,세종,세종시,지카바이러스감염증,0


In [ ]:
### 발생 저장

In [127]:
# df.to_csv('발생01062206_수정중_교수님틀.csv', index = False)

## 사망

In [12]:
사망.shape
사망.head()

(22491, 16)

,연도,시도,급수,병,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월
0,2001,강원,1급,에볼라바이러스병,-,-,-,-,-,-,-,-,-,-,-,-
1,2001,강원,1급,마버그열,-,-,-,-,-,-,-,-,-,-,-,-
2,2001,강원,1급,라싸열,-,-,-,-,-,-,-,-,-,-,-,-
3,2001,강원,1급,크리미안콩고출혈열,-,-,-,-,-,-,-,-,-,-,-,-
4,2001,강원,1급,남아메리카출혈열,-,-,-,-,-,-,-,-,-,-,-,-


In [85]:
사망.tail()

,연도,시도,급수,병,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월
22486,2021,충북,3급,진드기매개뇌염,0,0,0,0,0,0,0,0,0,0,0,0
22487,2021,충북,3급,유비저,0,0,0,0,0,0,0,0,0,0,0,0
22488,2021,충북,3급,치쿤구니야열,0,0,0,0,0,0,0,0,0,0,0,0
22489,2021,충북,3급,중증열성혈소판감소증후군(SFTS),0,0,0,0,0,0,0,0,0,0,0,0
22490,2021,충북,3급,지카바이러스감염증,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
# 사망 row : 연도(21) * 시도(17) * 병수(63)

사망T = 사망.iloc[:, 4:].T
사망T = 사망T.reset_index()
사망T.rename(columns = {'index':'월'}, inplace = True)

In [40]:
사망T

,월,0,1,2,3,4,5,6,7,8,...,22481,22482,22483,22484,22485,22486,22487,22488,22489,22490
0,1월,-,-,-,-,-,-,-,-,-,...,0,0,0,0,0,0,0,0,0,0
1,2월,-,-,-,-,-,-,-,-,-,...,0,0,0,0,0,0,0,0,0,0
2,3월,-,-,-,-,-,-,-,-,-,...,0,0,0,0,0,0,0,0,0,0
3,4월,-,-,-,-,-,-,-,-,-,...,0,0,0,0,0,0,0,0,0,0
4,5월,-,-,-,-,-,-,-,-,-,...,0,0,0,0,0,0,0,0,0,0
5,6월,-,-,-,-,-,-,-,-,-,...,0,0,0,0,0,0,0,0,0,0
6,7월,-,-,-,-,-,-,-,-,-,...,0,0,0,0,0,0,0,0,0,0
7,8월,-,-,-,-,-,-,-,-,-,...,0,0,0,0,0,0,0,0,0,0
8,9월,-,-,-,-,-,-,-,-,-,...,0,0,0,0,0,0,0,0,0,0
9,10월,-,-,-,-,-,-,-,-,-,...,0,0,0,0,0,0,0,0,0,0


In [83]:
df = pd.DataFrame()
for i in tqdm(range(사망T.shape[1])):
    
    ## 사망자수 열
    tmp월 = 사망T.iloc[:,1:].iloc[:,i]
    tmp월.index = [i for i in range(i*12,(i+1)*12)]
    
    tmp식별 = list(사망.iloc[i,:4].values)
    
    ## 식별열
    updt = []
    for j in range(1,13):
        updt.append(tmp식별+[f'{j}월'])
    tmp식별 = pd.DataFrame(updt)
    tmp식별.index = [k for k in range(i*12,(i+1)*12)]
    
#     tmp월
#     tmp식별
    tmpdf = pd.concat([tmp식별, tmp월], axis = 1)
    tmpdf.columns = ['연도', '시도', '급수', '병', '월', '사망수']
    
    df = pd.concat([df, tmpdf], ignore_index = True)

100%|███████████████████████████████████████████████████████████████████████████▉| 22491/22492 [06:03<00:00, 61.91it/s]


IndexError: single positional indexer is out-of-bounds

In [87]:
df.shape
21*17*63*12
## 에러는 났지만 잘 뽑힘.
df.head()
df.tail()

(269892, 6)

269892

,연도,시도,급수,병,월,사망수
0,2001,강원,1급,에볼라바이러스병,1월,-
1,2001,강원,1급,에볼라바이러스병,2월,-
2,2001,강원,1급,에볼라바이러스병,3월,-
3,2001,강원,1급,에볼라바이러스병,4월,-
4,2001,강원,1급,에볼라바이러스병,5월,-


,연도,시도,급수,병,월,사망수
269887,2021,충북,3급,지카바이러스감염증,8월,0
269888,2021,충북,3급,지카바이러스감염증,9월,0
269889,2021,충북,3급,지카바이러스감염증,10월,0
269890,2021,충북,3급,지카바이러스감염증,11월,0
269891,2021,충북,3급,지카바이러스감염증,12월,0


### 저장

In [96]:
# df.to_csv('사망0121_수정중_교수님틀.csv', index = False)

## 성별

In [42]:
성별.shape
성별.head()

(479808, 24)

,병,군구,연령성별,2001년,2002년,2003년,2004년,2005년,2006년,2007년,...,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
0,에볼라바이러스병,1,0~4세_남,-,-,-,-,-,-,-,...,0,0,0,0,0,0,0,0,0,0
1,에볼라바이러스병,1,0~4세_여,-,-,-,-,-,-,-,...,0,0,0,0,0,0,0,0,0,0
2,에볼라바이러스병,1,5~9세_남,-,-,-,-,-,-,-,...,0,0,0,0,0,0,0,0,0,0
3,에볼라바이러스병,1,5~9세_여,-,-,-,-,-,-,-,...,0,0,0,0,0,0,0,0,0,0
4,에볼라바이러스병,1,10~14세_남,-,-,-,-,-,-,-,...,0,0,0,0,0,0,0,0,0,0


In [56]:
성별.iloc[:28, :].tail()

,병,군구,연령성별,2001년,2002년,2003년,2004년,2005년,2006년,2007년,...,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
23,에볼라바이러스병,1,55~59세_여,-,-,-,-,-,-,-,...,0,0,0,0,0,0,0,0,0,0
24,에볼라바이러스병,1,60~64세_남,-,-,-,-,-,-,-,...,0,0,0,0,0,0,0,0,0,0
25,에볼라바이러스병,1,60~64세_여,-,-,-,-,-,-,-,...,0,0,0,0,0,0,0,0,0,0
26,에볼라바이러스병,1,65세 이상_남,-,-,-,-,-,-,-,...,0,0,0,0,0,0,0,0,0,0
27,에볼라바이러스병,1,65세 이상_여,-,-,-,-,-,-,-,...,0,0,0,0,0,0,0,0,0,0


### 교수님께서 말씀하신 형태로 만들기 위한 Transpose 및 그 이후의 작업.

In [78]:
연령성별연도별 = 성별.iloc[:, 2:].T
## colnms chng
연령성별연도별.columns = 연령성별연도별.iloc[0,:].values

## colnm된 맨 위 행 삭제
연령성별연도별.drop('연령성별', inplace = True)

## index가 연도였는데 column으로 빼주기
연령성별연도별 = 연령성별연도별.reset_index()

## 열 추가
연령성별연도별.rename(columns = {'index':'연도'}, inplace = True)

In [90]:
(연령성별연도별.shape[1]-1)/28 # T만 했을 때와 달리 열이 하나 늘어났기 때문에 ('연도'열) 하나 빼야함.

17136.0

In [82]:
연령성별연도별.head()

,0~4세_남,0~4세_여,5~9세_남,5~9세_여,10~14세_남,10~14세_여,15~19세_남,15~19세_여,20~24세_남,20~24세_여,...,45~49세_남,45~49세_여,50~54세_남,50~54세_여,55~59세_남,55~59세_여,60~64세_남,60~64세_여,65세 이상_남,65세 이상_여
2001년,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
2002년,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
2003년,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
2004년,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
2005년,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-


### 연산이 쓸데없이 복잡함.
- 걍 처음만들 때부터 이 틀로 해야하지 않았나 싶음.
- 두 번 실행하고 싶지는 않은.... 머리 아픈 시간이었다.

In [136]:
df = pd.DataFrame()
for i in tqdm(range(연령성별연도별.shape[1]//28)):
    ## 매 iter마다 data를 잇기 위해 이용할 tmpdf
    tmpdf = pd.DataFrame()
    
    ## 연도
    YY = 연령성별연도별.iloc[:,0]
    
    ## 발생자 수, 28개 col씩 끊기
    tmpval = 연령성별연도별.iloc[:,1:].iloc[:, i*28:(i+1)*28]
    
    ## 식별열
    idcols = 성별.iloc[i*28:(i+1)*28, :][['병', '군구']]
    tmp병 = pd.DataFrame(list(idcols.병.unique()) * tmpval.shape[0], columns = ['병'])
    tmp군구 = pd.DataFrame(list(idcols.군구.unique()) * tmpval.shape[0], columns = ['군구'])
    
    ## 현재 iter 옆으로 잇기
    tmpdf = pd.concat([tmpdf, YY, tmp병, tmp군구, tmpval], axis = 1)

    ## tmpdf를 df 아래로 내릴 때 에러 방지 위해 index 바꾸기
    tmpdf.index = list(range(i*tmpdf.shape[0], (i+1)*tmpdf.shape[0]))
    
    ## 밑으로 쌓아 과거의 데이터와 합치기
    df = pd.concat([df, tmpdf], axis = 0, ignore_index = True)

100%|████████████████████████████████████████████████████████████████████████████| 17136/17136 [57:01<00:00,  5.01it/s]


In [128]:
tmpdf.index
tmpdf

Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20],
      dtype='object')

,연도,병,군구,0~4세_남,0~4세_여,5~9세_남,5~9세_여,10~14세_남,10~14세_여,15~19세_남,...,45~49세_남,45~49세_여,50~54세_남,50~54세_여,55~59세_남,55~59세_여,60~64세_남,60~64세_여,65세 이상_남,65세 이상_여
0,2001년,에볼라바이러스병,1,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
1,2002년,에볼라바이러스병,1,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
2,2003년,에볼라바이러스병,1,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
3,2004년,에볼라바이러스병,1,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
4,2005년,에볼라바이러스병,1,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
5,2006년,에볼라바이러스병,1,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
6,2007년,에볼라바이러스병,1,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
7,2008년,에볼라바이러스병,1,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
8,2009년,에볼라바이러스병,1,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
9,2010년,에볼라바이러스병,1,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-


In [138]:
df.shape # (359856, 31)
df.head()

(359856, 31)

,연도,병,군구,0~4세_남,0~4세_여,5~9세_남,5~9세_여,10~14세_남,10~14세_여,15~19세_남,...,45~49세_남,45~49세_여,50~54세_남,50~54세_여,55~59세_남,55~59세_여,60~64세_남,60~64세_여,65세 이상_남,65세 이상_여
0,2001년,에볼라바이러스병,1,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
1,2002년,에볼라바이러스병,1,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
2,2003년,에볼라바이러스병,1,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
3,2004년,에볼라바이러스병,1,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
4,2005년,에볼라바이러스병,1,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-


In [139]:
df.tail()

,연도,병,군구,0~4세_남,0~4세_여,5~9세_남,5~9세_여,10~14세_남,10~14세_여,15~19세_남,...,45~49세_남,45~49세_여,50~54세_남,50~54세_여,55~59세_남,55~59세_여,60~64세_남,60~64세_여,65세 이상_남,65세 이상_여
359851,2017년,지카바이러스감염증,272,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
359852,2018년,지카바이러스감염증,272,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
359853,2019년,지카바이러스감염증,272,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
359854,2020년,지카바이러스감염증,272,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
359855,2021년,지카바이러스감염증,272,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 성별 저장

In [140]:
# df.to_csv('성별0121_수정중_교수님틀.csv', index = False)